In [ ]:
# Run once with real value then revert.
KEY = '<redacted>'


In [2]:
# Common imports, constants and functionality.

import datetime
import itertools
import json
import requests
import time

V1_BASE = 'https://api.torn.com'
V2_BASE = f'{V1_BASE}/v2'

class Missing:
    def _propagate(self):
        return self

    def __getattr__(self, name):
        return self._propagate
    
    def __bool__(self):
        return False


class Jsh:
    @classmethod
    def wrap(cls, inner):
        return Jsh(inner) if Jsh.wrappable(inner) else inner
    
    @classmethod
    def wrappable(cls, inner):
        return inner.__class__.__name__ == 'dict'
    
    @classmethod
    def fetch(cls, url):
        res = requests.get(url)
        if not res.ok:
            raise(RuntimeError(f'Failed to fetch "{url}": {res.error}'))
        data = json.loads(res.content)
        if 'error' in data:
            raise(RuntimeError(f'Error returned from "{url}": {data["error"]}'))
        return Jsh(data)
    
    def __init__(self, inner):
        assert(Jsh.wrappable(inner))
        self._inner = inner

    def __getattr__(self, name):
        if name == '_inner':
            return self._inner
        if not name in self._inner:
            return Missing()
        v = self._inner[name]
        match v.__class__.__name__:
            case 'dict': return Jsh(v)
            case 'list': return [Jsh.wrap(item) for item in v]
            case _: return v
    
    def __repr__(self):
        return str(self._inner)
    
    def has(self, name):
        return name in self._inner
    
    def getraw(self, name):
        return self._inner[name]
    
    def dumps(self):
        return json.dumps(self._inner, indent=2)


In [3]:
url = f'{V2_BASE}/racing/tracks?key={KEY}'
tracks = {}

try:
    jsh = Jsh.fetch(url)
except Exception as e:
    print(f'Failed to fetch "{url}": {e}')
    exit(1)
if not jsh.has('tracks'):
    print(f'No "tracks" in:\n{jsh.dumps()}')
    exit(1)
tracks.update([(track.id, track.title) for track in jsh.tracks])

print(f'Found {len(tracks)} tracks(s):')
for (id, name) in tracks.items():
    print(f'{name} [{id}]')


Found 16 tracks(s):
Uptown [6]
Withdrawal [7]
Underdog [8]
Parkland [9]
Docks [10]
Commerce [11]
Two Islands [12]
Industrial [15]
Vector [16]
Mudpit [17]
Hammerhead [18]
Sewage [19]
Meltdown [20]
Speedway [21]
Stone Park [23]
Convict [24]


In [ ]:
url = f'{V2_BASE}/racing/carupgrades?key={KEY}'

try:
    jsh = Jsh.fetch(url)
except Exception as e:
    print(f'Failed to fetch "{url}": {e}')
    exit(1)
if not jsh.has('carupgrades'):
    print(f'No "carupgrades" in:\n{jsh.dumps()}')
    exit(1)

print('id,name,description,category,subcategory,class_required,points,money,top_speed,acceleration,braking,handling,safety,dirt,tarmac')
for u in jsh.carupgrades:
    print(f'{u.id},{u.name},"{u.description}",{u.category},{u.subcategory},{u.class_required},{u.cost.points},{u.cost.cash},{u.effects.top_speed},{u.effects.acceleration},{u.effects.braking},'
          f'{u.effects.handling},{u.effects.safety},{u.effects.dirt},{u.effects.tarmac}')


id,name,description,category,subcategory,class_required,points,money,top_speed,acceleration,braking,handling,safety,dirt,tarmac
1,Strip Out,"By removing all of the unnecessary interior from the car you can increase the power to weight ratio, therefore increasing speed and acceleration. The car will also handle better and brake quicker as there is less mass to maneuver and stop.",Weight Reduction,Strip out,E,1,0,1,1,1,1,0,0,0
4,Uprated Springs (Soft),"These replace the original springs and allow for improved handling off road.",Suspension,Springs,E,2,3500,0,0,0,1,0,1,-1
5,Uprated Springs (Medium),"These replace the original springs and allow for slightly improved handling on all surfaces.",Suspension,Springs,E,2,3500,0,0,0,1,0,0,0
6,Uprated Springs (Hard),"These replace the original springs and allow for improved handling on a track.",Suspension,Springs,E,2,3500,0,0,0,1,0,-1,1
9,Uprated Brake Pads,"These pads will allow you to brake slightly later due to greater friction properties than

In [4]:
url = f'{V2_BASE}/faction/members?key={KEY}'
members = {}

try:
    jsh = Jsh.fetch(url)
except Exception as e:
    print(f'Failed to fetch "{url}": {e}')
    exit(1)
if not jsh.has('members'):
    print(f'No "members" in:\n{jsh.dumps()}')
    exit(1)
members.update([(member.id, member.name) for member in jsh.members])

print(f'Found {len(members)} members(s)')

for (id, name) in members.items():
    print(f'{name} [{id}]')


Found 30 members(s)
Big_Tex1429 [1513734]
Mingle [2122084]
Talendrife [2143613]
MaxamillionXX7 [2293468]
Lexii [2637832]
Pvpkiller [2682292]
DukeSilver [2733982]
Sofro [2734960]
Socharis [2801249]
AUSTINEZ [2843077]
Rhysand [2919031]
Kitana1988 [3037508]
Popcorn- [3040244]
Sniffles666 [3249261]
Giftedbasicbee [3360473]
Shodgson [3509557]
VeronicaMars [3562079]
Fikowi [3602025]
Schwartzenadder [3623941]
plexar [3646316]
ChriSabi [3681518]
SweetPetite [3713328]
rsty [3743574]
NTHRITE [3799007]
Boduki [3825722]
000Machie000 [3829798]
Djiinni [3871156]
Parzival175 [3873916]
Ahmad1Albustami [3894597]
jordankk [3946191]


In [13]:
url = f'{V2_BASE}/racing/races?cat=custom&limit=100'
cutoff = datetime.datetime(2025, 11, 8, 0, 0, 0).timestamp()
remaining = 50
done = False

while url and not done:
    url += f'&key={KEY}'
    try:
        jsh = Jsh.fetch(url)
    except Exception as e:
        print(f'Failed to fetch "{url}": {e}')
        raise(e)
    if not jsh.has('races'):
        print(f'No "races" in:\n{jsh.dumps()}')
        raise(RuntimeError(f'No "races" in:\n{jsh.dumps()}'))
    for race in jsh.races:
        if race.schedule.start < cutoff:
          done = True
          break
        if race.status == 'in_progress' and race.track_id == 10: # Docks
            print(f'{race.id}: {race.title} @ {tracks[race.track_id]} ({race.status})')
            remaining -= 1
        if remaining < 1:
            done = True
            break
    url = jsh._metadata.links.prev


17511890: Socks XP @ Docks (in_progress)
17512308: xprace @ Docks (in_progress)
17512445: bobbybob1694&#039;s race @ Docks (in_progress)
17512896: 1hr Start - Docks @ Docks (in_progress)
17512849: Matthew715&#039;s race @ Docks (in_progress)
17512815: DarkManV&#039;s race @ Docks (in_progress)
17512773: 1hr Start - Docks @ Docks (in_progress)
17512743: Hospitalize&#039;s race @ Docks (in_progress)
17512687: Santereas&#039;s race @ Docks (in_progress)
17512663: SisterChristian&#039;s race @ Docks (in_progress)
17512559: Ice_Addict&#039;s race @ Docks (in_progress)
17512318: Alba_Gu_Brath&#039;s race @ Docks (in_progress)
17512434: 1hr Start - Docks @ Docks (in_progress)
17512451: kurapylly&#039;s race @ Docks (in_progress)
17512384: mozgurl80&#039;s race @ Docks (in_progress)
17512366: Mnixos&#039;s race @ Docks (in_progress)
17511810: Letcherinium&#039;s race @ Docks (in_progress)
17512355: Buffalo_Bill&#039;s race @ Docks (in_progress)
17512280: Docks 100l @ Docks (in_progress)
175122

In [16]:
rids = [16894378, 16981530, 17057677, 17134189, 17282712, 17351216, 17426344, 17500628]
n = len(rids)
hdrs1 = list(itertools.repeat("", 2*n+1))
hdrs2 = ["name"]
for i in range(0, n):
    hdrs2.extend(["pos", "time"])
results = dict(
    (id, list(itertools.chain([name], itertools.repeat("", 2*len(rids)))))
    for (id, name) in members.items()
)
for i, rid in enumerate(rids):
    url = f'{V2_BASE}/racing/{rid}/race?key={KEY}'
    try:
        jsh = Jsh.fetch(url)
    except Exception as e:
        print(f'Failed to fetch "{url}": {e}')
        exit(1)
    if not jsh.has('race'):
        print(f'No "race" in:\n{jsh.dumps()}')
        exit(1)
    hdrs1[2*i+1] = tracks[jsh.race.track_id]
    for res in jsh.race.results:
        if res.driver_id in members:
            results[res.driver_id][2*i+1:2*i+3] = [
                str(res.position or ""),
                "DNF" if res.has_crashed else str(res.race_time or ""),
            ]

print(','.join(hdrs1))
print(','.join(hdrs2))
for res in results.values():
    print(','.join(res))


,Two Islands,,Stone Park,,Industrial,,Hammerhead,,Underdog,,Mudpit,,Withdrawal,,Docks,
name,pos,time,pos,time,pos,time,pos,time,pos,time,pos,time,pos,time,pos,time
Big_Tex1429,9,12201.91,10,8552.09,3,8539.91,10,6856.58,,,,,3,12555.92,4,17882.53
Mingle,2,10975.75,2,7988.65,1,8509.96,2,6117.43,2,9443.78,2,3964.55,,,3,17855.48
Talendrife,,,,,,,,,,,,,,,,
MaxamillionXX7,,,,,,,,,,,,,,,,
Lexii,3,11070.75,4,8046.13,6,8658.94,8,6806.63,3,9479.06,3,3966.66,2,12546.7,1,17801.2
Pvpkiller,13,25738.68,16,18027.42,14,14050.49,17,12512.64,14,15996.58,,,10,21377.13,,
DukeSilver,6,11683.71,9,8407.59,5,8632.41,,,6,9511.47,11,6203.86,6,12694.51,7,17998.81
Sofro,14,26635.15,14,14926.47,13,13901.66,16,11663.99,13,15719.78,13,11353.49,9,18990.89,9,26516.01
Socharis,1,10961.4,1,7962.49,4,8558.44,1,6082.04,,,,,1,12477.77,,
AUSTINEZ,,,,,,,15,9853.12,,,,,,,,
Rhysand,,,,,,,,,,,,,,,,
Kitana1988,,,5,8307.52,,,4,6259.57,7,9523.39,5,4006.9,,,6,17998.46
Popcorn-,,,,,,,,,,,,,,,,
Sniffles666,,,,,,,,,,,,,,,,
Giftedbasicb

In [ ]:
url = f'{V2_BASE}/faction/chains?limit=100&sort=DESC'
cutoff = datetime.datetime(2025, 10, 24, 0, 0, 0)
chains = []

while url:
    url += f'&key={KEY}'
    try:
        jsh = Jsh.fetch(url)
    except Exception as e:
        print(f'Failed to fetch "{url}": {e}')
        exit(1)
    if not jsh.has('chains'):
        print(f'No "chains" in:\n{jsh.dumps()}')
        exit(1)
    chains += jsh.chains
    url = jsh._metadata.links.prev

print(f'Found {len(chains)} chain(s) in total')

(c, s, r) = itertools.repeat(0, 3)
for chain in chains:
    if chain.start < cutoff.timestamp():
        break
    c += 1
    s += chain.chain
    r += chain.respect

print(f'{c} chain(s) totaling {s} hit(s) and {r} respect found after {cutoff.strftime("%c")}')


Found 882 chain(s) in total
149 chain(s) totaling 13645 hit(s) and 34567.81 respect found after Fri Oct 24 00:00:00 2025


In [3]:
url = f'{V1_BASE}/torn/?selections=stocks&key={KEY}'
try:
  jsh = Jsh.fetch(url)
except Exception as e:
  print(f'Failed to fetch "{url}": {e}')
  exit(1)
if not jsh.has('stocks'):
  print(f'No "stocks" in:\n{jsh.dumps()}')
  exit(1)

print('Ticker,Name,Benefit,Type,Frequency,Requirement,Price,Cost')
for v in jsh.getraw("stocks").values():
  s = Jsh.wrap(v)
  print(f'{s.acronym},"{s.name}","{s.benefit.description}",{s.benefit.type},{s.benefit.frequency},{s.benefit.requirement},{s.current_price},'
        f'{s.current_price*s.benefit.requirement}')


Ticker,Name,Benefit,Type,Frequency,Requirement,Price,Cost
TSB,"Torn & Shanghai Banking","$50,000,000",active,31,3000000,1182.54,3547620000.0
TCI,"Torn City Investments","a 10% bank interest bonus",passive,7,1500000,1194.2,1791300000.0
SYS,"Syscore MFG","an Advanced firewall",passive,7,3000000,684.88,2054640000.0
LAG,"Legal Authorities Group","1x Lawyer's Business Card",active,7,750000,452.95,339712500.0
IOU,"Insured On Us","$12,000,000",active,31,3000000,179.05,537150000.0
GRN,"Grain","$4,000,000",active,31,500000,305.53,152765000.0
THS,"Torn City Health Service","1x Box of Medical Supplies",active,7,150000,383.31,57496500.0
YAZ,"Yazoo","Free banner advertising",passive,7,1000000,55.04,55040000.0
TCT,"The Torn City Times","$1,000,000",active,31,100000,316.29,31629000.000000004
CNC,"Crude & Co","$80,000,000",active,31,7500000,889.62,6672150000.0
MSG,"Messaging Inc.","Free classified advertising",passive,7,300000,282.67,84801000.0
TMI,"TC Music Industries","$25,000,000",active,31,6000000